In [100]:
import sys
sys.path.append('..')
from economic_brazil.treinamento.treinamento_algoritimos import carregar
from economic_brazil.analisando_modelos.analise_modelos_regressao import MetricasModelosDicionario,PredicaosModelos
from economic_brazil.analisando_modelos.regressao_conformal import ConformalRegressionPlotter
from economic_brazil.predicao_valores_futuros import KerasTrainedRegressor, Predicao
import pandas as pd
from joblib import parallel_config
import warnings
import pickle
import os
warnings.filterwarnings("ignore", category=UserWarning)
path_codigos_rodando = os.path.join(os.getcwd())

##########################################################################################################DEFININDO VARIAVEIS############################################################################################################

variavel_predicao = 'pib'

##########################################################################################CARREGANDO DADOS############################################################################################################

arquivo_pib = path_codigos_rodando+f'/avaliacao_modelos/apresentacao_streamlit/dados_treinamento_{variavel_predicao}.pkl'
dados_carregados = pickle.load(open(arquivo_pib, 'rb'))

dados = dados_carregados['dados']
x_treino = dados_carregados['x_treino']
x_teste = dados_carregados['x_teste']
y_treino = dados_carregados['y_treino']
y_teste = dados_carregados['y_teste']
if 'pca' in dados_carregados.keys():
    pca = dados_carregados['pca']
if 'scaler' in dados_carregados.keys():    
    scaler = dados_carregados['scaler']
data_divisao_treino_teste = dados_carregados['data_divisao_treino_teste']
tratando = dados_carregados['tratando']

##########################################################################################################CARREGANDO MODELOS############################################################################################################

modelos_carregados = carregar(diretorio=path_codigos_rodando+f'/modelos_salvos/modelos_{variavel_predicao}/',gradiente_boosting=True, xg_boost=True, cat_boost=True, regressao_linear=True, redes_neurais=True, sarimax=False)


##########################################################################################################PREVENDO OS DADOS DE TREINO E TESTE################################################################################
predi = PredicaosModelos(modelos_carregados, x_treino, y_treino, x_teste, y_teste)
predicoes_treino, predicoes_teste = predi.predicoes()
x_treino, y_treino, x_teste, y_teste, x_treino_recorrente, y_treino_recorrente, x_teste_recorrente, y_teste_recorrente = predi.return_dados()

###########################################################################################################################CALCULANDO METRICAS################################################################################
metri = MetricasModelosDicionario()
metrica_teste = metri.calculando_metricas(predicoes_teste, y_teste, y_teste_recorrente)
metrica_treino = metri.calculando_metricas(predicoes_treino, y_treino, y_treino_recorrente)

print(metrica_teste)
print(metrica_treino)

#metrica_teste.to_csv("/workspaces/Predicoes_macroeconomicas/codigos_rodando/avaliacao_modelos/metricas_teste.csv")
#metrica_treino.to_csv("/workspaces/Predicoes_macroeconomicas/codigos_rodando/avaliacao_modelos/metricas_treino.csv")

index_treino = dados[dados.index <= data_divisao_treino_teste][6:].index
index_teste = dados[dados.index > data_divisao_treino_teste][4:].index

#############################################################################################################PLOTANDO METRICAS################################################################################
"""metri.plotando_predicoes(
    y_treino,
    predicoes_treino,
    index=index_treino,
    title="Predições nos dados de treino",
    save=True,
    diretorio=path_codigos_rodando+f'/avaliacao_modelos/predicao_treino_{variavel_predicao}.png',
)

metri.plotando_predicoes(
    y_teste,
    predicoes_teste,
    index=index_teste,
    title="Predições nos dados de teste",
    save=True,
    diretorio=path_codigos_rodando+f'/avaliacao_modelos/predicao_teste_{variavel_predicao}.png',
)

metri.plotando_predicoes_go_treino_teste(
    y_treino,
    y_teste,
    predicoes_treino,
    predicoes_teste,
    index_treino,
    index_teste,
    save=True,
    diretorio=path_codigos_rodando+f'/avaliacao_modelos/predicao_treino_teste_{variavel_predicao}.png',
    type_arquivo='png'
)"""

################################################################################################AVALIANDO MODELOS################################################################################
modelos_validos_teste = metrica_teste[metrica_teste['Variance'] > 0.1]

if modelos_validos_teste.empty:
    melhor_modelo = metrica_teste['Variance'].idxmax()
    print(f'Melhor modelo baseado na Variance mais alta, com valor de {metrica_teste["Variance"].max()}:',melhor_modelo)
else:
    melhor_modelo = modelos_validos_teste['MAE'].idxmin()
    print(f'Melhor modelo baseado na MAE mais baixo, com valor de {metrica_teste["MAE"].min()}:',melhor_modelo)

#Conformal
index_treino_conformal = dados[dados.index <= data_divisao_treino_teste].index
index_teste_conformal = dados[dados.index > data_divisao_treino_teste].index

with parallel_config(backend='threading', n_jobs=2):
    if melhor_modelo == 'redes_neurais':
        conformal = ConformalRegressionPlotter(KerasTrainedRegressor(modelos_carregados[melhor_modelo]), x_treino_recorrente, x_teste_recorrente, y_treino[1:], y_teste[1:])
        y_pred, y_pis, _,_ = conformal.regressao_conformal()
        conformal.plot_prediction_intervals(index_train=index_treino_conformal, index_test=index_teste_conformal,title=f'Predição Intervals {melhor_modelo}',save=True,diretorio=path_codigos_rodando+f'/avaliacao_modelos/regressao_conforma_teste_{variavel_predicao}.png')
    else:
        conformal = ConformalRegressionPlotter(modelos_carregados[melhor_modelo], x_treino, x_teste, y_treino, y_teste)
        y_pred, y_pis, _,_ = conformal.regressao_conformal()
        conformal.plot_prediction_intervals(index_train=index_treino_conformal, index_test=index_teste_conformal,title=f'Predição Intervals {melhor_modelo}',save=True,diretorio=path_codigos_rodando+f'/avaliacao_modelos/regressao_conforma_teste_{variavel_predicao}.png')

dados_corformal = {
    'data': index_teste[-1],
    'predicao': y_pred,
    'intervalo_lower': y_pis.squeeze()[0:,0],
    'intervalo_upper': y_pis.squeeze()[0:,1],}    
## Predicao futuro
with parallel_config(backend='threading', n_jobs=2):
    predicao = Predicao(x_treino,y_treino,tratando,dados,melhor_modelo,modelos_carregados[melhor_modelo],periodo='Trimestral',coluna=variavel_predicao)
    
dados_predicao_futuro, _, index_futuro = predicao.criando_dados_futuros()
dados_predicao = predicao.criando_dataframe_predicoes()
#dados_predicao.to_csv('/workspaces/Predicoes_macroeconomicas/codigos_rodando/avaliacao_modelos/dados_predicao.csv',index=False)
print(dados_predicao)
data_predicao,intervalo_lower,intervalo_upper,predicao_proximo_mes = predicao.predicao_ultimo_periodo()
dados_futuro = {
    'data': data_predicao,
    'intervalo_lower': intervalo_lower,
    'intervalo_upper': intervalo_upper,
    'predicao': predicao_proximo_mes}
print(f'Data da predição:{data_predicao}, Valor da predição:{predicao_proximo_mes}, Intervalo de predição [lower,upper] :{intervalo_lower,intervalo_upper}')
predicao.plotando_predicoes(save=True,diretorio=path_codigos_rodando+f'/avaliacao_modelos/predicao_futuro_{variavel_predicao}.png')

###salvando os dados
dados_salvos = {}
dados_salvos['dados'] = dados
dados_salvos['y_treino'] = y_treino
dados_salvos['x_treino'] = x_treino
dados_salvos['y_treino_recorrente'] = y_treino_recorrente
dados_salvos['x_treino_recorrente'] = x_treino_recorrente
dados_salvos['x_teste'] = x_teste
dados_salvos['y_teste'] = y_teste
dados_salvos['y_teste_recorrente'] = y_teste_recorrente
dados_salvos['x_teste_recorrente'] = x_teste_recorrente
dados_salvos['metrica_teste'] = metrica_teste
dados_salvos['metrica_treino'] = metrica_treino
dados_salvos['predicoes_treino'] = predicoes_treino
dados_salvos['predicoes_teste'] = predicoes_teste
dados_salvos['index_treino'] = index_treino
dados_salvos['index_teste'] = index_teste
dados_salvos['melhor_modelo'] = melhor_modelo
dados_salvos['dados_conformal'] = dados_corformal
dados_salvos['dados_predicao'] = dados_predicao
dados_salvos['dados_futuro'] = dados_futuro
dados_salvos['modelos_carregados'] = list(modelos_carregados.keys())

with open(path_codigos_rodando+f'/avaliacao_modelos/apresentacao_streamlit/dados_salvos_{variavel_predicao}.pkl', 'wb') as f:
    pickle.dump(dados_salvos, f)
    
print('Avaliacao concluida com sucesso!')


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
                     MAE    MSE  RMSE     R²  Variance
xg_boost            0.55   0.39  0.62  -0.62      0.00
cat_boost           0.57   0.46  0.68  -0.94      0.00
gradiente_boosting  0.89   1.05  1.03  -3.42      0.00
regressao_linear    1.19   1.96  1.40  -7.22      0.38
redes_neurais       3.34  11.46  3.39 -50.08      0.27
                     MAE   MSE  RMSE    R²  Variance
regressao_linear    0.78  0.93  0.97  0.37      0.76
gradiente_boosting  0.90  1.28  1.13  0.14      0.06
cat_boost           0.91  1.35  1.16  0.09      0.02
xg_boost            0.92  1.39  1.18  0.07      0.01
redes_neurais       0.95  1.41  1.19  0.07      1.14
Melhor modelo baseado na MAE mais baixo, com valor de 0.55: regressao_linear
Coverage and prediction interval width mean for CV+: 0.000, 4.868
Coverage and prediction interval width mean for CV+: 0.000, 8.628
            intervalo_lower  intervalo_upper  predicao
2022-10-01 

In [98]:
import sys
sys.path.append('..')
from economic_brazil.coleta_dados.economic_data_brazil import EconomicBrazil
from economic_brazil.processando_dados.tratando_dados import TratandoDados
from economic_brazil.treinamento.treinamento_algoritimos import TreinandoModelos
import warnings
import pandas as pd
import pickle
import os

path_codigos_rodando = os.path.join(os.getcwd())

warnings.filterwarnings("ignore", category=UserWarning)

banco_central_codes = {
    "indice_condicoes_economicas": 4394,
    "indice_valores_garantias_imoveis_residencias_financiados": 21340,
    "venda_veiculos_concessionarias": 1378,
    "indicador_movimento_comercio_prazo": 1453,
    "indice_volume_vendas_varejo": 1455,
    "imposto_sobre_produtos": 22098
}

variaveis_ibge = {
    "ipca": {
        "codigo": '1737',
        "territorial_level": "1",
        "ibge_territorial_code": "all",
        "variable": "63",
    },
    "custo_m2": {
        "codigo": '2296',
        "territorial_level": "1",
        "ibge_territorial_code": "all",
        "variable": "1198",
    },
    "pmc_volume": {
        "codigo": '8186',
        "territorial_level": "1",
        "ibge_territorial_code": "all",
        "variable": "11709",
    },
    "producao_fisica_industrial": {
        "codigo": '8159',
        "territorial_level": "1",
        "ibge_territorial_code": "all",
        "variable": "11599",},

    "producao_fisica_para_construcao_civil": {
        "codigo": '7980',
        "territorial_level": "1",
        "ibge_territorial_code": "all",
        "variable": "11599",},
    
    "producao_soja_milho": {    
        "codigo": '6588',
        "territorial_level": "1",
        "ibge_territorial_code": "all",
        "variable": "35",},
    
    "precos_construcao_civil": {
        "codigo": '2296',
        "territorial_level": "1",
        "ibge_territorial_code": "all",
        "variable": "1198",},
    
    "volume_servicos_(pms)": {
        "codigo": '8162',
        "territorial_level": "1",
        "ibge_territorial_code": "all",
        "variable": "11621",
    }
    
    
}

codigos_ipeadata_padrao = {
    "rendimento_real_medio": "PNADC12_RRTH12",
    "pessoas_forca_trabalho": "PNADC12_FT12",
    "taxa_desocupacao": "PNADC12_TDESOC12",
    "caged_novo": "CAGED12_SALDON12",
    "caged_antigo": "CAGED12_SALDO12",
    'ibovespa': 'ANBIMA12_IBVSP12',
    "taja_juros_ltn": "ANBIMA12_TJTLN1212",
    'exportacoes': 'PAN12_XTV12',
    'importacoes': 'PAN12_MTV12',
    'm_1': 'BM12_M1MN12',
    'taxa_cambio': 'PAN12_ERV12',
    'atividade_economica': 'SGS12_IBCBR12',
}

lista = [
    'seguro desemprego']

indicadores_ibge_link = {
    "pib": "https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela5932.xlsx&terr=N&rank=-&query=t/5932/n1/all/v/6564/p/all/c11255/90707/d/v6564%201/l/v,p,t%2Bc11255&verUFs=false&verComplementos2=false&verComplementos1=false&omitirIndentacao=false&abreviarRotulos=false&exibirNotas=false&agruparNoCabecalho=false",
    "despesas_publica": "https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela5932.xlsx&terr=N&rank=-&query=t/5932/n1/all/v/6561/p/all/c11255/93405/d/v6561%201/l/v,p%2Bc11255,t",
    "capital_fixo": "https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela5932.xlsx&terr=N&rank=-&query=t/5932/n1/all/v/6561/p/all/c11255/93406/d/v6561%201/l/v,p%2Bc11255,t",
    "producao_industrial_manufatureira": "https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela8158.xlsx&terr=N&rank=-&query=t/8158/n1/all/v/11599/p/all/c543/129278/d/v11599%205/l/v,p%2Bc543,t",
    'soja' :'https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela6588.xlsx&terr=N&rank=-&query=t/6588/n1/all/v/35/p/all/c48/0,39443/l/v,p%2Bc48,t',
    'milho_1' :'https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela6588.xlsx&terr=N&rank=-&query=t/6588/n1/all/v/35/p/all/c48/0,39441/l/v,p%2Bc48,t',
    'milho_2' :'https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela6588.xlsx&terr=N&rank=-&query=t/6588/n1/all/v/35/p/all/c48/0,39442/l/v,p%2Bc48,t',
    'pms': 'https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela8162.xlsx&terr=N&rank=-&query=t/8162/n1/all/v/11622/p/all/c11046/56726/c12355/107071/d/v11622%205/l/v,p%2Bc11046,t%2Bc12355'
}

codigos_fred = {'nasdaq100':'NASDAQ100',
                'taxa_cambio_efetiva':'RBBRBIS',
                'cboe_nasdaq':'VXNCLS',
                'taxa_juros_interbancaria':'IRSTCI01BRM156N',
                'atividade_economica_eua':'USPHCI',
                'indice_confianca_manufatura':'BSCICP03BRM665S',
                'indice_confianca_exportadores':'BSXRLV02BRM086S',
                'indice_tendencia_emprego':'BRABREMFT02STSAM',
                'indice_confianca_consumidor':'CSCICP03BRM665S',
                'capacidade_instalada':'BSCURT02BRM160S',}

############################################################################## VARIAVEL PREDICAO ################################################################
variavel_predicao = 'pib'


################################################################################ DADOS ############################################################################

dados_bcb = True
dados_ibge = True
dados_expectativas_inflacao = False
dados_metas_inflacao = False
dados_ibge_link = True
dados_ipeadata = True
dados_google_trends = True
dados_fred =True

economic_brasil = EconomicBrazil(codigos_banco_central=banco_central_codes, 
                       codigos_ibge=variaveis_ibge,
                       codigos_ipeadata=codigos_ipeadata_padrao,
                       lista_termos_google_trends=lista,
                       codigos_ibge_link=indicadores_ibge_link,
                       codigos_fred=codigos_fred,
                       data_inicio="1997-01-01")

dados = economic_brasil.dados_brazil(
                dados_bcb=dados_bcb,
                dados_ipeadata=dados_ipeadata,
                dados_fred=dados_fred,
                dados_metas_inflacao=dados_metas_inflacao,
                dados_google_trends=dados_google_trends,
                dados_expectativas_inflacao=dados_expectativas_inflacao,
                dados_ibge_link=dados_ibge_link,
                dados_ibge_codigos=dados_ibge,
    
)
#Colcando os dados em trimestre
dados_trimestral = dados.resample('QS').median()

############################################################################## TRATAMENTO ############################################################################


tratando_scaler = True
tratando_pca = True
tratando_dummy_covid = True
tratando_defasagens = True
tratando_datas = True
tratando_estacionaridade = True

data_divisao = '2020-06-01'

tratando = TratandoDados(dados_trimestral,
                         coluna_label=variavel_predicao,
                         data_divisao=data_divisao,
                         n_components=10,
                         numero_defasagens=4)

x_treino, x_teste, y_treino, y_teste,pca, scaler = tratando.tratando_dados(scaler=tratando_scaler, 
                                                                           pca=tratando_pca, 
                                                                           covid=tratando_dummy_covid, 
                                                                           datas=tratando_datas, 
                                                                           defasagens=tratando_defasagens, 
                                                                           estacionaridade=tratando_estacionaridade,
                                                                           )

data_divisao_treino_teste = tratando.data_divisao_treino_teste()

param_grid_gradiente = {
                    "n_estimators": [2, 10,20,30, 50, 100, 200, 300, 500],
                    "learning_rate": [0.1, 0.15, 0.2, 0.4, 0.6, 0.8, 1, 1.25, 1.5,1.75],
                    "max_depth": [1, 3, 5, 7, 9],
                }

param_grid_xgboost = {
                    "n_estimators": [2, 10,20,30, 50, 100, 200, 300, 500],
                    "learning_rate": [0.1, 0.15, 0.2, 0.4, 0.6, 0.8, 1, 1.25, 1.5,1.75],
                    "max_depth": [1, 3, 5, 7, 9],
                }

param_grid_catboost = {
                    "iterations": [2, 10,20,30, 50, 100, 200, 300, 500],
                    "learning_rate": [0.1, 0.15, 0.2, 0.4, 0.6, 0.8, 1, 1.25, 1.5,1.75],
                    "depth": [1, 3, 5, 7, 9,10,15],
                }


tuning = TreinandoModelos(x_treino, 
                          y_treino, 
                          x_teste, 
                          y_teste,
                          numero_divisoes = 5,
                          diretorio=f'../codigos_rodando/modelos_salvos/modelos_{variavel_predicao}/',
                          salvar_modelo=True)

modelo_redes_neurais = True
modelo_cast = True
modelo_sarimax = False
modelo_gradient_boosting = True
modelo_regresao_linear = True
modelo_xgboost = True
redes_neurais_tuning = False

modelos_tunning = tuning.treinar_modelos(redes_neurais=modelo_redes_neurais,
                                         cat_boost=modelo_cast,
                                         sarimax=modelo_sarimax,
                                         gradiente_boosting=modelo_gradient_boosting,
                                         regressao_linear=modelo_regresao_linear,
                                         xg_boost=modelo_xgboost,
                                         redes_neurais_tuning=redes_neurais_tuning,
                                         param_grid_catboost=param_grid_catboost,
                                        param_grid_xgboost=param_grid_xgboost,
                                        param_grid_gradiente=param_grid_gradiente,
                                         )


Valores validos apartir de 2012-09-01 00:00:00 para a variável Custo médio m² - variação percentual em doze meses
Erro na coleta de dados da variável pmc_volume. Verifique se os códigos {'codigo': '8186', 'territorial_level': '1', 'ibge_territorial_code': 'all', 'variable': '11709'} estão ativos em https://sidra.ibge.gov.br/home/pms/brasil.
Erro na coleta de dados da variável producao_soja_milho. Verifique se os códigos {'codigo': '6588', 'territorial_level': '1', 'ibge_territorial_code': 'all', 'variable': '35'} estão ativos em https://sidra.ibge.gov.br/home/pms/brasil.
Valores validos apartir de 2012-09-01 00:00:00 para a variável Custo médio m² - variação percentual em doze meses
Erro na coleta de dados da variável volume_servicos_(pms). Verifique se os códigos {'codigo': '8162', 'territorial_level': '1', 'ibge_territorial_code': 'all', 'variable': '11621'} estão ativos em https://sidra.ibge.gov.br/home/pms/brasil.
Too many requests error for seguro desemprego. Skipping this term fo

In [99]:
param_grid_gradiente = {
                    "n_estimators": [2, 10,20,30, 50, 100, 200, 300, 500],
                    "learning_rate": [0.1, 0.15, 0.2, 0.4, 0.6, 0.8, 1, 1.25, 1.5,1.75],
                    "max_depth": [1, 3, 5, 7, 9],
                }

param_grid_xgboost = {
                    "n_estimators": [2, 10,20,30, 50, 100, 200, 300, 500],
                    "learning_rate": [0.1, 0.15, 0.2, 0.4, 0.6, 0.8, 1, 1.25, 1.5,1.75],
                    "max_depth": [1, 3, 5, 7, 9],
                }

param_grid_catboost = {
                    "iterations": [2, 10,20,30, 50, 100, 200, 300, 500],
                    "learning_rate": [0.1, 0.15, 0.2, 0.4, 0.6, 0.8, 1, 1.25, 1.5,1.75],
                    "depth": [1, 3, 5, 7, 9,10,15],
                }


tuning = TreinandoModelos(x_treino, 
                          y_treino, 
                          x_teste, 
                          y_teste,
                          numero_divisoes = 5,
                          diretorio=f'../codigos_rodando/modelos_salvos/modelos_{variavel_predicao}/',
                          salvar_modelo=True)

modelo_redes_neurais = True
modelo_cast = True
modelo_sarimax = False
modelo_gradient_boosting = True
modelo_regresao_linear = True
modelo_xgboost = True
redes_neurais_tuning = False

modelos_tunning = tuning.treinar_modelos(redes_neurais=modelo_redes_neurais,
                                         cat_boost=modelo_cast,
                                         sarimax=modelo_sarimax,
                                         gradiente_boosting=modelo_gradient_boosting,
                                         regressao_linear=modelo_regresao_linear,
                                         xg_boost=modelo_xgboost,
                                         redes_neurais_tuning=redes_neurais_tuning,
                                         param_grid_catboost=param_grid_catboost,
                                        param_grid_xgboost=param_grid_xgboost,
                                        param_grid_gradiente=param_grid_gradiente
                                         )


Best Bayesian Optimization Params: OrderedDict([('learning_rate', 0.1), ('max_depth', 5), ('n_estimators', 2)]) Score: -0.9384839424053901
Modelo Gradiente Boosting Treinado
Best Bayesian Optimization Params: OrderedDict([('learning_rate', 0.1), ('max_depth', 1), ('n_estimators', 2)]) Score: -0.9915975000858307
Modelo XG Boost Treinado
0:	learn: 0.6896583	total: 11.3ms	remaining: 1.12s
1:	learn: 0.4915033	total: 16.1ms	remaining: 787ms
2:	learn: 0.3508006	total: 25.3ms	remaining: 819ms
3:	learn: 0.2581455	total: 26.4ms	remaining: 633ms
4:	learn: 0.1780585	total: 41.1ms	remaining: 782ms
5:	learn: 0.1223573	total: 45.7ms	remaining: 717ms
6:	learn: 0.0841384	total: 47.9ms	remaining: 637ms
7:	learn: 0.0578452	total: 60ms	remaining: 690ms
8:	learn: 0.0397694	total: 66.6ms	remaining: 673ms
9:	learn: 0.0273415	total: 74.3ms	remaining: 669ms
10:	learn: 0.0187973	total: 77.1ms	remaining: 624ms
11:	learn: 0.0129231	total: 87.2ms	remaining: 639ms


learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.


12:	learn: 0.0088846	total: 95.9ms	remaining: 642ms
13:	learn: 0.0061082	total: 103ms	remaining: 634ms
14:	learn: 0.0041994	total: 108ms	remaining: 613ms
15:	learn: 0.0028871	total: 118ms	remaining: 617ms
16:	learn: 0.0019849	total: 126ms	remaining: 614ms
17:	learn: 0.0013646	total: 134ms	remaining: 608ms
18:	learn: 0.0009382	total: 138ms	remaining: 586ms
19:	learn: 0.0006450	total: 141ms	remaining: 565ms
20:	learn: 0.0004434	total: 145ms	remaining: 546ms
21:	learn: 0.0003049	total: 150ms	remaining: 531ms
22:	learn: 0.0002096	total: 154ms	remaining: 517ms
23:	learn: 0.0001441	total: 158ms	remaining: 501ms
24:	learn: 0.0000991	total: 164ms	remaining: 491ms
25:	learn: 0.0000681	total: 167ms	remaining: 476ms
26:	learn: 0.0000468	total: 169ms	remaining: 457ms
27:	learn: 0.0000322	total: 176ms	remaining: 452ms
28:	learn: 0.0000221	total: 180ms	remaining: 440ms
29:	learn: 0.0000152	total: 182ms	remaining: 425ms
30:	learn: 0.0000105	total: 186ms	remaining: 414ms
31:	learn: 0.0000072	total: 18

learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.


6:	learn: 0.1091014	total: 107ms	remaining: 1.42s
7:	learn: 0.0778321	total: 108ms	remaining: 1.24s
8:	learn: 0.0535096	total: 112ms	remaining: 1.13s
9:	learn: 0.0367878	total: 116ms	remaining: 1.04s
10:	learn: 0.0252916	total: 121ms	remaining: 980ms
11:	learn: 0.0173880	total: 125ms	remaining: 916ms
12:	learn: 0.0119543	total: 128ms	remaining: 858ms
13:	learn: 0.0082185	total: 134ms	remaining: 822ms
14:	learn: 0.0056503	total: 139ms	remaining: 786ms
15:	learn: 0.0038845	total: 142ms	remaining: 746ms
16:	learn: 0.0026706	total: 147ms	remaining: 715ms
17:	learn: 0.0018361	total: 151ms	remaining: 687ms
18:	learn: 0.0012623	total: 155ms	remaining: 660ms
19:	learn: 0.0008678	total: 159ms	remaining: 635ms
20:	learn: 0.0005966	total: 163ms	remaining: 613ms
21:	learn: 0.0004102	total: 167ms	remaining: 593ms
22:	learn: 0.0002820	total: 171ms	remaining: 571ms
23:	learn: 0.0001939	total: 175ms	remaining: 553ms
24:	learn: 0.0001333	total: 178ms	remaining: 533ms
25:	learn: 0.0000916	total: 181ms	r

...

33:	learn: 0.3061754	total: 7.48ms	remaining: 36.5ms
34:	learn: 0.2924226	total: 7.65ms	remaining: 36.1ms
35:	learn: 0.2884761	total: 7.85ms	remaining: 35.7ms
36:	learn: 0.2832980	total: 8.13ms	remaining: 35.8ms
37:	learn: 0.2728601	total: 8.34ms	remaining: 35.6ms
38:	learn: 0.2670906	total: 8.59ms	remaining: 35.5ms
39:	learn: 0.2584159	total: 8.81ms	remaining: 35.3ms
40:	learn: 0.2519525	total: 9.1ms	remaining: 35.3ms
41:	learn: 0.2448435	total: 9.29ms	remaining: 34.9ms
42:	learn: 0.2396765	total: 9.55ms	remaining: 34.9ms
43:	learn: 0.2360939	total: 9.77ms	remaining: 34.6ms
44:	learn: 0.2307263	total: 10ms	remaining: 34.4ms
45:	learn: 0.2228665	total: 10.2ms	remaining: 34.1ms
46:	learn: 0.2194979	total: 10.4ms	remaining: 33.8ms
47:	learn: 0.2158531	total: 10.5ms	remaining: 33.4ms
48:	learn: 0.2119769	total: 10.7ms	remaining: 33ms
49:	learn: 0.2077967	total: 10.8ms	remaining: 32.5ms
50:	learn: 0.2000997	total: 11ms	remaining: 32.2ms
51:	learn: 0.1949783	total: 11.2ms	remaining: 32ms
52

learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.


0:	learn: 0.9266228	total: 249us	remaining: 249us
1:	learn: 0.8248376	total: 13.6ms	remaining: 0us
0:	learn: 1.0635682	total: 258us	remaining: 258us
1:	learn: 0.9650693	total: 843us	remaining: 0us


learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.


0:	learn: 1.0855967	total: 227us	remaining: 227us
1:	learn: 1.0184615	total: 407us	remaining: 0us
0:	learn: 1.0863775	total: 160us	remaining: 160us
1:	learn: 1.0031666	total: 302us	remaining: 0us
0:	learn: 1.1514307	total: 231us	remaining: 231us
1:	learn: 1.0843532	total: 333us	remaining: 0us
0:	learn: 0.9543044	total: 381us	remaining: 381us
1:	learn: 0.9067304	total: 591us	remaining: 0us
0:	learn: 1.1760015	total: 378us	remaining: 378us
1:	learn: 1.0989398	total: 593us	remaining: 0us
0:	learn: 1.1505774	total: 698us	remaining: 698us
1:	learn: 1.1018058	total: 1.33ms	remaining: 0us
0:	learn: 1.1129753	total: 313us	remaining: 313us
1:	learn: 1.0693716	total: 511us	remaining: 0us
0:	learn: 1.1999782	total: 329us	remaining: 329us
1:	learn: 1.1529174	total: 559us	remaining: 0us
0:	learn: 0.9351803	total: 4.65ms	remaining: 4.65ms
1:	learn: 0.8821386	total: 6.01ms	remaining: 0us
0:	learn: 1.1531967	total: 1.35ms	remaining: 1.35ms
1:	learn: 1.0881860	total: 1.99ms	remaining: 0us
0:	learn: 1.1

In [74]:
dados.isnull().sum().all().all()

False

In [82]:
pd.DataFrame(x_teste).notnull().all().all()

True

In [83]:
pd.DataFrame(y_teste).notnull().all().all()

True

In [84]:
pd.DataFrame(x_treino).notnull().all().all()

True

In [85]:
pd.DataFrame(y_treino).notnull().all().all()

True